In [0]:
spark.conf.set(
  "fs.azure.sas.retail.adfproject2rishustorage.dfs.core.windows.net",
  "sp=racwl&st=2025-12-14T06:53:27Z&se=2025-12-14T15:08:27Z&spr=https&sv=2024-11-04&sr=c&sig=UF%2BqS5qOe6gkTBEsDJjiV80gdmpl2MXZsMj92qW6fVo%3D"
)


In [0]:
dbutils.fs.ls("abfss://retail@adfproject2rishustorage.dfs.core.windows.net/bronze/")


[FileInfo(path='abfss://retail@adfproject2rishustorage.dfs.core.windows.net/bronze/customer/', name='customer/', size=0, modificationTime=1765655769000),
 FileInfo(path='abfss://retail@adfproject2rishustorage.dfs.core.windows.net/bronze/product/', name='product/', size=0, modificationTime=1765655763000),
 FileInfo(path='abfss://retail@adfproject2rishustorage.dfs.core.windows.net/bronze/store/', name='store/', size=0, modificationTime=1765699306000),
 FileInfo(path='abfss://retail@adfproject2rishustorage.dfs.core.windows.net/bronze/transaction/', name='transaction/', size=0, modificationTime=1765655739000)]

In [0]:
bronze_path = "abfss://retail@adfproject2rishustorage.dfs.core.windows.net/bronze/"
silver_path = "abfss://retail@adfproject2rishustorage.dfs.core.windows.net/silver/"
gold_path   = "abfss://retail@adfproject2rishustorage.dfs.core.windows.net/gold/"


In [0]:
from pyspark.sql import DataFrame

def load_layer(layer_path: str) -> dict:
    """Load all subfolders in a layer into DataFrames"""
    dfs = {}
    for folder_info in dbutils.fs.ls(layer_path):
        folder_name = folder_info.name.strip("/")
        folder_path = layer_path + folder_name + "/"
        
        # List Parquet files
        files = [f for f in dbutils.fs.ls(folder_path) if f.name.endswith(".parquet") and f.size > 0]
        if not files:
            print(f"No Parquet files in {folder_name}, skipping...")
            continue
        
        # Load all Parquet files
        df = spark.read.format("parquet").load(folder_path + "*.parquet")
        dfs[folder_name] = df
        print(f"{folder_name} loaded with {df.count()} rows")
    return dfs


In [0]:
# Bronze
bronze_dfs = load_layer(bronze_path)

# Silver
silver_dfs = load_layer(silver_path)

# Gold
gold_dfs = load_layer(gold_path)


No Parquet files in customer, skipping...
product loaded with 10 rows
store loaded with 5 rows
transaction loaded with 30 rows


In [0]:
display(bronze_dfs["product"].limit(5))
display(bronze_dfs["store"].limit(5))
display(bronze_dfs["transaction"].limit(5))


product_id,product_name,category,price
1,Wireless Mouse,Electronics,799.99
2,Bluetooth Speaker,Electronics,1299.49
3,Yoga Mat,Fitness,499.00
4,Laptop Stand,Accessories,999.99
5,Notebook Set,Stationery,149.00


store_id,store_name,location
1,City Mall Store,Mumbai
2,High Street Store,Delhi
3,Tech World Outlet,Bangalore
4,Downtown Mini Store,Pune
5,Mega Plaza,Chennai


transaction_id,customer_id,product_id,store_id,quantity,transaction_date
1,127,8,4,4,2025-03-31
2,105,3,4,5,2024-11-12
3,116,2,2,3,2025-05-01
4,120,8,1,1,2024-11-02
5,105,5,2,1,2025-03-17


In [0]:
df_product = spark.read.format("parquet") \
    .load("abfss://retail@adfproject2rishustorage.dfs.core.windows.net/bronze/product/*.parquet")


In [0]:
# Product
df_product = spark.read.format("parquet").load(
    "abfss://retail@adfproject2rishustorage.dfs.core.windows.net/bronze/product/*.parquet"
)
display(df_product)

# Store
df_store = spark.read.format("parquet").load(
    "abfss://retail@adfproject2rishustorage.dfs.core.windows.net/bronze/store/*.parquet"
)
display(df_store)

# Transaction
df_transaction = spark.read.format("parquet").load(
    "abfss://retail@adfproject2rishustorage.dfs.core.windows.net/bronze/transaction/*.parquet"
)
display(df_transaction)


product_id,product_name,category,price
1,Wireless Mouse,Electronics,799.99
2,Bluetooth Speaker,Electronics,1299.49
3,Yoga Mat,Fitness,499.00
4,Laptop Stand,Accessories,999.99
5,Notebook Set,Stationery,149.00
6,Water Bottle,Fitness,299.00
7,Smartwatch,Electronics,4999.00
8,Desk Organizer,Accessories,399.00
9,Dumbbell Set,Fitness,1999.00
10,Pen Drive 32GB,Electronics,599.00


store_id,store_name,location
1,City Mall Store,Mumbai
2,High Street Store,Delhi
3,Tech World Outlet,Bangalore
4,Downtown Mini Store,Pune
5,Mega Plaza,Chennai


transaction_id,customer_id,product_id,store_id,quantity,transaction_date
1,127,8,4,4,2025-03-31
2,105,3,4,5,2024-11-12
3,116,2,2,3,2025-05-01
4,120,8,1,1,2024-11-02
5,105,5,2,1,2025-03-17
6,110,7,3,5,2025-01-04
7,110,7,2,5,2025-01-01
8,126,7,5,2,2025-06-08
9,123,1,3,2,2024-10-08
10,124,2,2,5,2024-08-27


In [0]:

df_product.show(5)
df_store.show(5)
df_transaction.show(5)


+----------+-----------------+-----------+-------+
|product_id|     product_name|   category|  price|
+----------+-----------------+-----------+-------+
|         1|   Wireless Mouse|Electronics| 799.99|
|         2|Bluetooth Speaker|Electronics|1299.49|
|         3|         Yoga Mat|    Fitness| 499.00|
|         4|     Laptop Stand|Accessories| 999.99|
|         5|     Notebook Set| Stationery| 149.00|
+----------+-----------------+-----------+-------+
only showing top 5 rows
+--------+-------------------+---------+
|store_id|         store_name| location|
+--------+-------------------+---------+
|       1|    City Mall Store|   Mumbai|
|       2|  High Street Store|    Delhi|
|       3|  Tech World Outlet|Bangalore|
|       4|Downtown Mini Store|     Pune|
|       5|         Mega Plaza|  Chennai|
+--------+-------------------+---------+

+--------------+-----------+----------+--------+--------+----------------+
|transaction_id|customer_id|product_id|store_id|quantity|transaction_dat

In [0]:
df_product.printSchema()
df_store.printSchema()
df_transaction.printSchema()


root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: decimal(10,2) (nullable = true)

root
 |-- store_id: integer (nullable = true)
 |-- store_name: string (nullable = true)
 |-- location: string (nullable = true)

root
 |-- transaction_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- transaction_date: date (nullable = true)



In [0]:
dfs = {
    "product": df_product,
    "store": df_store,
    "transaction": df_transaction
}


In [0]:
df_sales = df_transaction.join(df_product, "product_id").join(df_store, "store_id")
df_sales.show(10)


+--------+----------+--------------+-----------+--------+----------------+-----------------+-----------+-------+-------------------+---------+
|store_id|product_id|transaction_id|customer_id|quantity|transaction_date|     product_name|   category|  price|         store_name| location|
+--------+----------+--------------+-----------+--------+----------------+-----------------+-----------+-------+-------------------+---------+
|       4|         8|             1|        127|       4|      2025-03-31|   Desk Organizer|Accessories| 399.00|Downtown Mini Store|     Pune|
|       4|         3|             2|        105|       5|      2024-11-12|         Yoga Mat|    Fitness| 499.00|Downtown Mini Store|     Pune|
|       2|         2|             3|        116|       3|      2025-05-01|Bluetooth Speaker|Electronics|1299.49|  High Street Store|    Delhi|
|       1|         8|             4|        120|       1|      2024-11-02|   Desk Organizer|Accessories| 399.00|    City Mall Store|   Mumbai|

In [0]:
customer_path = "abfss://retail@adfproject2rishustorage.dfs.core.windows.net/bronze/customer/manish040596/azure-data-engineer---multi-source/refs/heads/main/customers.parquet"

df_customer = spark.read.format("parquet").load(customer_path)
display(df_customer)


customer_id,first_name,last_name,email,phone,city,registration_date
101,Ravi,Yadav,user101@example.com,9887654321,Delhi,2023-09-14
102,Nina,Joshi,user102@example.com,9876543210,Mumbai,2024-01-21
103,Sonal,Sharma,user103@example.com,9865432109,Bangalore,2023-07-10
104,Karan,Patel,user104@example.com,9854321098,Hyderabad,2024-02-05
105,Riya,Singh,user105@example.com,9843210987,Chennai,2023-06-28
106,Ajay,Mishra,user106@example.com,9832109876,Pune,2024-03-10
107,Priya,Kapoor,user107@example.com,9821098765,Ahmedabad,2023-05-12
108,Rahul,Verma,user108@example.com,9810987654,Kolkata,2023-08-19
109,Pooja,Mehta,user109@example.com,9809876543,Delhi,2024-04-01
110,Deepak,Nair,user110@example.com,9798765432,Mumbai,2023-10-14


## Now work on Silver layer(data cleaning and all this stuffs)

In [0]:
from pyspark.sql.functions import col

df_transaction = df_transaction.select(
    col("transaction_id").cast("int"),
    col("customer_id").cast("int"),
    col("product_id").cast("int"),
    col("store_id").cast("int"),
    col("quantity").cast("int"),
    col("transaction_date").cast("date")
)


In [0]:
from pyspark.sql.functions import col, trim
from pyspark.sql import DataFrame

# --- Define silver path ---
silver_path = "abfss://retail@adfproject2rishustorage.dfs.core.windows.net/silver/"

# -----------------------------
# 1. Transactions
# -----------------------------
df_transaction_silver = df_transaction.select(
    col("transaction_id").cast("int"),
    col("customer_id").cast("int"),
    col("product_id").cast("int"),
    col("store_id").cast("int"),
    col("quantity").cast("int"),
    col("transaction_date").cast("date")
).na.drop(subset=["transaction_id","customer_id","product_id","store_id"]) \
 .dropDuplicates(["transaction_id"])

# Save to silver
df_transaction_silver.write.mode("overwrite").parquet(silver_path + "transaction/")

# -----------------------------
# 2. Products
# -----------------------------
df_product_silver = df_product.select(
    col("product_id").cast("int"),
    trim(col("product_name")).alias("product_name"),
    trim(col("category")).alias("category"),
    col("price").cast("double")
).dropDuplicates(["product_id"])

# Save to silver
df_product_silver.write.mode("overwrite").parquet(silver_path + "product/")

# -----------------------------
# 3. Stores
# -----------------------------
df_store_silver = df_store.select(
    col("store_id").cast("int"),
    trim(col("store_name")).alias("store_name"),
    trim(col("location")).alias("location")
).dropDuplicates(["store_id"])

# Save to silver
df_store_silver.write.mode("overwrite").parquet(silver_path + "store/")

# -----------------------------
# 4. Customers
# -----------------------------
df_customer_silver = df_customer.select(
    col("customer_id").cast("int"),
    trim(col("first_name")).alias("first_name"),
    trim(col("last_name")).alias("last_name"),
    trim(col("email")).alias("email"),
    trim(col("city")).alias("city"),
    col("registration_date").cast("date")
).dropDuplicates(["customer_id"])

# Save to silver
df_customer_silver.write.mode("overwrite").parquet(silver_path + "customer/")

# -----------------------------
# 5. Optional: Display top rows
# -----------------------------
display(df_transaction_silver.limit(5))
display(df_product_silver.limit(5))
display(df_store_silver.limit(5))
display(df_customer_silver.limit(5))


transaction_id,customer_id,product_id,store_id,quantity,transaction_date
28,101,7,5,3,2024-11-15
26,126,6,2,2,2024-09-21
27,127,8,1,1,2024-10-10
12,108,5,1,4,2025-05-26
22,107,9,3,4,2024-11-16


product_id,product_name,category,price
1,Wireless Mouse,Electronics,799.99
6,Water Bottle,Fitness,299.0
3,Yoga Mat,Fitness,499.0
5,Notebook Set,Stationery,149.0
9,Dumbbell Set,Fitness,1999.0


store_id,store_name,location
1,City Mall Store,Mumbai
2,High Street Store,Delhi
3,Tech World Outlet,Bangalore
4,Downtown Mini Store,Pune
5,Mega Plaza,Chennai


customer_id,first_name,last_name,email,city,registration_date
148,Mahesh,Patel,user148@example.com,Kolkata,2024-02-15
137,Shweta,Mehta,user137@example.com,Chennai,2023-05-06
133,Ishaan,Verma,user133@example.com,Delhi,2023-07-03
108,Rahul,Verma,user108@example.com,Kolkata,2023-08-19
101,Ravi,Yadav,user101@example.com,Delhi,2023-09-14


In [0]:
from pyspark.sql.functions import col

# Join all silver tables
df_silver = df_transaction_silver \
    .join(df_customer_silver, "customer_id") \
    .join(df_product_silver, "product_id") \
    .join(df_store_silver, "store_id") \
    .withColumn("total_amount", col("quantity") * col("price"))

# Preview joined data
display(df_silver.limit(10))
df_silver.printSchema()


store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,city,registration_date,product_name,category,price,store_name,location,total_amount
1,5,108,12,4,2025-05-26,Rahul,Verma,user108@example.com,Kolkata,2023-08-19,Notebook Set,Stationery,149.0,City Mall Store,Mumbai,596.0
5,7,101,28,3,2024-11-15,Ravi,Yadav,user101@example.com,Delhi,2023-09-14,Smartwatch,Electronics,4999.0,Mega Plaza,Chennai,14997.0
5,7,126,8,2,2025-06-08,Bhavna,Nair,user126@example.com,Mumbai,2023-07-17,Smartwatch,Electronics,4999.0,Mega Plaza,Chennai,9998.0
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,Bangalore,2023-07-10,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,2399.9700000000003
4,9,122,23,2,2025-04-30,Tina,Kapoor,user122@example.com,Pune,2023-09-20,Dumbbell Set,Fitness,1999.0,Downtown Mini Store,Pune,3998.0
4,1,120,14,5,2024-07-17,Alka,Mishra,user120@example.com,Hyderabad,2023-12-01,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,3999.95
4,8,127,1,4,2025-03-31,Rohit,Mehta,user127@example.com,Bangalore,2023-08-30,Desk Organizer,Accessories,399.0,Downtown Mini Store,Pune,1596.0
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999.0,Tech World Outlet,Bangalore,7996.0
1,5,130,20,2,2024-07-30,Tanvi,Kapoor,user130@example.com,Pune,2023-12-22,Notebook Set,Stationery,149.0,City Mall Store,Mumbai,298.0
3,1,102,11,2,2024-08-11,Nina,Joshi,user102@example.com,Mumbai,2024-01-21,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,1599.98


root
 |-- store_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- transaction_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- city: string (nullable = true)
 |-- registration_date: date (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: double (nullable = true)
 |-- store_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [0]:
silver_path = "abfss://retail@adfproject2rishustorage.dfs.core.windows.net/silver/transactions_joined/"


In [0]:
df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .save(silver_path)


In [0]:
df_silver_read = spark.read.format("delta").load(silver_path)


In [0]:
display(df_silver_read)


store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,city,registration_date,product_name,category,price,store_name,location,total_amount
1,5,108,12,4,2025-05-26,Rahul,Verma,user108@example.com,Kolkata,2023-08-19,Notebook Set,Stationery,149.0,City Mall Store,Mumbai,596.0
5,7,101,28,3,2024-11-15,Ravi,Yadav,user101@example.com,Delhi,2023-09-14,Smartwatch,Electronics,4999.0,Mega Plaza,Chennai,14997.0
5,7,126,8,2,2025-06-08,Bhavna,Nair,user126@example.com,Mumbai,2023-07-17,Smartwatch,Electronics,4999.0,Mega Plaza,Chennai,9998.0
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,Bangalore,2023-07-10,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,2399.9700000000003
4,9,122,23,2,2025-04-30,Tina,Kapoor,user122@example.com,Pune,2023-09-20,Dumbbell Set,Fitness,1999.0,Downtown Mini Store,Pune,3998.0
4,1,120,14,5,2024-07-17,Alka,Mishra,user120@example.com,Hyderabad,2023-12-01,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,3999.95
4,8,127,1,4,2025-03-31,Rohit,Mehta,user127@example.com,Bangalore,2023-08-30,Desk Organizer,Accessories,399.0,Downtown Mini Store,Pune,1596.0
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999.0,Tech World Outlet,Bangalore,7996.0
1,5,130,20,2,2024-07-30,Tanvi,Kapoor,user130@example.com,Pune,2023-12-22,Notebook Set,Stationery,149.0,City Mall Store,Mumbai,298.0
3,1,102,11,2,2024-08-11,Nina,Joshi,user102@example.com,Mumbai,2024-01-21,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,1599.98


In [0]:
# DBTITLE 1,Dump joined silver data to ADLS as Delta
silver_path = "/mnt/retail_project/silver/transactions_joined/"

df_silver.write.mode("overwrite").format("delta").save(silver_path)


In [0]:
%sql

CREATE DATABASE IF NOT EXISTS rishuprojectdatabrick.silver;


In [0]:
# Using Spark DataFrame API
df_silver_loaded = spark.read.format("delta").load(silver_path)
display(df_silver_loaded)

# Using SQL
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS silver.transactions_joined
    USING DELTA
    LOCATION '{silver_path}'
 """)


store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,city,registration_date,product_name,category,price,store_name,location,total_amount
1,5,108,12,4,2025-05-26,Rahul,Verma,user108@example.com,Kolkata,2023-08-19,Notebook Set,Stationery,149.0,City Mall Store,Mumbai,596.0
5,7,101,28,3,2024-11-15,Ravi,Yadav,user101@example.com,Delhi,2023-09-14,Smartwatch,Electronics,4999.0,Mega Plaza,Chennai,14997.0
5,7,126,8,2,2025-06-08,Bhavna,Nair,user126@example.com,Mumbai,2023-07-17,Smartwatch,Electronics,4999.0,Mega Plaza,Chennai,9998.0
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,Bangalore,2023-07-10,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,2399.9700000000003
4,9,122,23,2,2025-04-30,Tina,Kapoor,user122@example.com,Pune,2023-09-20,Dumbbell Set,Fitness,1999.0,Downtown Mini Store,Pune,3998.0
4,1,120,14,5,2024-07-17,Alka,Mishra,user120@example.com,Hyderabad,2023-12-01,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,3999.95
4,8,127,1,4,2025-03-31,Rohit,Mehta,user127@example.com,Bangalore,2023-08-30,Desk Organizer,Accessories,399.0,Downtown Mini Store,Pune,1596.0
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999.0,Tech World Outlet,Bangalore,7996.0
1,5,130,20,2,2024-07-30,Tanvi,Kapoor,user130@example.com,Pune,2023-12-22,Notebook Set,Stationery,149.0,City Mall Store,Mumbai,298.0
3,1,102,11,2,2024-08-11,Nina,Joshi,user102@example.com,Mumbai,2024-01-21,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,1599.98


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8661538280959833>, line 6
      3 display(df_silver_loaded)
      5 # Using SQL
----> 6 spark.sql(f"""
      7     CREATE TABLE IF NOT EXISTS silver.transactions_joined
      8     USING DELTA
      9     LOCATION '{silver_path}'
     10 """)

File /databricks/spark/python/pyspark/sql/connect/session.py:879, in SparkSession.sql(self, sqlQuery, args, **kwargs)
    876         _views.append(SubqueryAlias(df._plan, name))
    878 cmd = SQL(sqlQuery, _args, _named_args, _views)
--> 879 data, properties, ei = self.client.execute_command(cmd.command(self._client))
    880 if "sql_command_result" in properties:
    881     df = DataFrame(CachedRelation(properties["sql_command_result"]), self)

File /databricks/spark/python/pyspark/sql/connect/client/core.py:1589, in SparkConnectClient.execute_command(self, command, observations, 

In [0]:
silver_path = "abfss://retail@adfproject2rishustorage.dfs.core.windows.net/silver/transactions_joined/"
gold_path = "abfss://retail@adfproject2rishustorage.dfs.core.windows.net/gold/sales_summary/"


In [0]:
silver_df = spark.read.format("delta").load(silver_path)


In [0]:
display(silver_df)


store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,city,registration_date,product_name,category,price,store_name,location,total_amount
1,5,108,12,4,2025-05-26,Rahul,Verma,user108@example.com,Kolkata,2023-08-19,Notebook Set,Stationery,149.0,City Mall Store,Mumbai,596.0
5,7,101,28,3,2024-11-15,Ravi,Yadav,user101@example.com,Delhi,2023-09-14,Smartwatch,Electronics,4999.0,Mega Plaza,Chennai,14997.0
5,7,126,8,2,2025-06-08,Bhavna,Nair,user126@example.com,Mumbai,2023-07-17,Smartwatch,Electronics,4999.0,Mega Plaza,Chennai,9998.0
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,Bangalore,2023-07-10,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,2399.9700000000003
4,9,122,23,2,2025-04-30,Tina,Kapoor,user122@example.com,Pune,2023-09-20,Dumbbell Set,Fitness,1999.0,Downtown Mini Store,Pune,3998.0
4,1,120,14,5,2024-07-17,Alka,Mishra,user120@example.com,Hyderabad,2023-12-01,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,3999.95
4,8,127,1,4,2025-03-31,Rohit,Mehta,user127@example.com,Bangalore,2023-08-30,Desk Organizer,Accessories,399.0,Downtown Mini Store,Pune,1596.0
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999.0,Tech World Outlet,Bangalore,7996.0
1,5,130,20,2,2024-07-30,Tanvi,Kapoor,user130@example.com,Pune,2023-12-22,Notebook Set,Stationery,149.0,City Mall Store,Mumbai,298.0
3,1,102,11,2,2024-08-11,Nina,Joshi,user102@example.com,Mumbai,2024-01-21,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,1599.98


In [0]:
from pyspark.sql.functions import sum, countDistinct, avg


In [0]:
gold_df = silver_df.groupBy(
    "transaction_date",
    "product_id",
    "product_name",
    "category",
    "store_id",
    "store_name",
    "location"
).agg(
    sum("quantity").alias("total_quantity_sold"),
    sum("total_amount").alias("total_sales_amount"),
    countDistinct("transaction_id").alias("number_of_transactions"),
    avg("total_amount").alias("average_transaction_value")
)


In [0]:
display(gold_df)


transaction_date,product_id,product_name,category,store_id,store_name,location,total_quantity_sold,total_sales_amount,number_of_transactions,average_transaction_value
2024-11-02,8,Desk Organizer,Accessories,1,City Mall Store,Mumbai,1,399.0,1,399.0
2024-08-11,1,Wireless Mouse,Electronics,3,Tech World Outlet,Bangalore,2,1599.98,1,1599.98
2024-12-13,8,Desk Organizer,Accessories,4,Downtown Mini Store,Pune,5,1995.0,1,1995.0
2025-05-04,3,Yoga Mat,Fitness,3,Tech World Outlet,Bangalore,4,1996.0,1,1996.0
2025-05-26,5,Notebook Set,Stationery,1,City Mall Store,Mumbai,4,596.0,1,596.0
2024-07-14,1,Wireless Mouse,Electronics,5,Mega Plaza,Chennai,1,799.99,1,799.99
2024-07-17,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,5,3999.95,1,3999.95
2024-09-05,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,3,2399.9700000000003,1,2399.9700000000003
2025-06-03,9,Dumbbell Set,Fitness,4,Downtown Mini Store,Pune,2,3998.0,1,3998.0
2024-08-27,2,Bluetooth Speaker,Electronics,2,High Street Store,Delhi,5,6497.45,1,6497.45


In [0]:
gold_df.printSchema()


root
 |-- transaction_date: date (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- store_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- total_quantity_sold: long (nullable = true)
 |-- total_sales_amount: double (nullable = true)
 |-- number_of_transactions: long (nullable = false)
 |-- average_transaction_value: double (nullable = true)



In [0]:
gold_df.write \
    .format("delta") \
    .mode("overwrite") \
    .save(gold_path)


In [0]:
gold_df_read = spark.read.format("delta").load(gold_path)


In [0]:
display(gold_df_read)


transaction_date,product_id,product_name,category,store_id,store_name,location,total_quantity_sold,total_sales_amount,number_of_transactions,average_transaction_value
2024-11-02,8,Desk Organizer,Accessories,1,City Mall Store,Mumbai,1,399.0,1,399.0
2024-08-11,1,Wireless Mouse,Electronics,3,Tech World Outlet,Bangalore,2,1599.98,1,1599.98
2024-12-13,8,Desk Organizer,Accessories,4,Downtown Mini Store,Pune,5,1995.0,1,1995.0
2025-05-04,3,Yoga Mat,Fitness,3,Tech World Outlet,Bangalore,4,1996.0,1,1996.0
2025-05-26,5,Notebook Set,Stationery,1,City Mall Store,Mumbai,4,596.0,1,596.0
2024-07-14,1,Wireless Mouse,Electronics,5,Mega Plaza,Chennai,1,799.99,1,799.99
2024-07-17,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,5,3999.95,1,3999.95
2024-09-05,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,3,2399.9700000000003,1,2399.9700000000003
2025-06-03,9,Dumbbell Set,Fitness,4,Downtown Mini Store,Pune,2,3998.0,1,3998.0
2024-08-27,2,Bluetooth Speaker,Electronics,2,High Street Store,Delhi,5,6497.45,1,6497.45
